<a href="https://colab.research.google.com/github/Untick/InspectrumClinic_RS_gr1/blob/main/Marfida%20Alexander/A_Marfida_stajirovka_profpatolog_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Стажировка. Профпатолог (INSPECTRUM CLINIC)
- Анализ и обработка исходных данных

## Загрузка исходных данных

In [ ]:
# Подключение библиотек

# Работа с массивами данных
import numpy as np

# Работа с табличными данными
import pandas as pd

# Функции-утилиты для работы с категориальными данными
from tensorflow.keras import utils

# Класс для конструирования последовательной модели нейронной сети
from tensorflow.keras.models import Sequential, Model

# Основные слои
from tensorflow.keras.layers import Dense, Dropout, SpatialDropout1D, BatchNormalization, Embedding, Flatten, Activation, Input, concatenate
from tensorflow.keras.layers import SimpleRNN, GRU, LSTM, Bidirectional, Conv1D, MaxPooling1D, GlobalMaxPooling1D

# Оптимизаторы
from tensorflow.keras.optimizers import Adam, Adadelta, SGD, Adagrad, RMSprop

# Токенизатор для преобразования текстов в последовательности
from tensorflow.keras.preprocessing.text import Tokenizer

# Масштабирование данных
from sklearn.preprocessing import StandardScaler

# Загрузка датасетов из облака google
import gdown

# Регулярные выражения
import re

# Отрисовка графиков
import matplotlib.pyplot as plt
plt.style.use('dark_background')

# Создание статистических графиков
import seaborn as sns

# Табличные данные
from fastai.tabular import *

# Метрики для расчета ошибок
from sklearn.metrics import mean_squared_error, mean_absolute_error

%matplotlib inline

In [ ]:
##
# gdown.download('https://storage.yandexcloud.net/aiueducation/Content/base/l9/wine_base.csv', None, quiet=True)
gdown.download('https://storage.yandexcloud.net/terratraineeship/23_InspectrumClinic_RS/datasets/%D0%B4%D0%B0%D1%82%D0%B0%D1%81%D0%B5%D1%82%20%D0%BF%D1%80%D0%BE%D1%84%D0%BF%D0%B0%D1%82%D0%BE%D0%BB%D0%BE%D0%B3%20%D1%87%D0%B0%D1%81%D1%82%D1%8C.xlsx', None, quiet=True)


'%D0%B4%D0%B0%D1%82%D0%B0%D1%81%D0%B5%D1%82%20%D0%BF%D1%80%D0%BE%D1%84%D0%BF%D0%B0%D1%82%D0%BE%D0%BB%D0%BE%D0%B3%20%D1%87%D0%B0%D1%81%D1%82%D1%8C.xlsx'

Загруженный excel переименован в ICdata.xlsx

In [ ]:
# Монтирование Goodle-диска
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Чтение исходного excel в переменную
# df_backup = pd.read_excel('ICdata.xlsx')
df_backup = pd.read_excel('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/ICdata.xlsx')
# df_backup = pd.read_excel('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/ICdata_I(71).xlsx')

In [ ]:
# Запись или чтение последней версии обработанных исходных данных
# df.to_excel('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/ICtemp.xlsx', , index=False)
#df_backup.to_excel('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/ICtemp.xlsx', index=False) #!#, encoding='utf-8-sig')
    #!# /usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
    #!# return func(*args, **kwargs)
df_backup = pd.read_excel('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/ICtemt.xlsx')

In [ ]:
# Создание рабочей копии датафрейма
df = df_backup.copy() # (deep=True) - default

## Служебные функции

### Функция просмотра содержания ненулевых уникальных значений полей в выбранных столбцах

In [ ]:
def prn_unique_txt_col(df, *arg, sort_asc = False):
  for colname in arg:
    column_list = list(df.loc[~df[colname].isnull(), colname].unique())
    count_list = len(column_list)
    print(f'*** {colname}({count_list} уникальных): ***')
    to_join = sorted(column_list) if sort_asc else column_list
    print('\n'.join(to_join))
    print()

### Функция подсчёта заполненности столбцов

In [ ]:
# Аргументы - параметры
def count_columns_fill(df, *arg):
  print('Текущая структура датафрейма: ', df.shape, '\n')
  for colname in arg:
    print(f'*** {colname}: ***')
    print('Количество непустых элементов: ', len(df[~df[colname].isna()]) )
    print('Количество уникальных элементов: ', len(df.loc[~df[colname].isnull(), colname].unique()), '\n')

In [ ]:
# Аргументы - список
def count_columns_in_list_fill(df, arg_list):
  count_list = 0
  print('Текущая структура датафрейма: ', df.shape, '\n')
  for colname in arg_list:
    count_list += 1
    print(f'*** {colname}: ***')
    print('Количество непустых элементов: ', len(df[~df[colname].isna()]) )
    print('Количество уникальных элементов: ', len(df.loc[~df[colname].isnull(), colname].unique()), '\n')
  print('Количество столбцов: ', count_list)

### Функция удаления из списка малоинформативных столбцов

In [ ]:
def reduce_colomns_unique_1(df, dfcols_to_save):
  ret = []
  print('Входящий размер списка столбцов: ', len(dfcols_to_save))
  for colname in dfcols_to_save:
    if len(df.loc[~df[colname].isnull(), colname].unique()) != 1:
      ret.append(colname)
  print('Исходящий размер списка столбцов: ', len(ret))
  ret
  return ret

### Функция сравнения пары списков на совпадение

In [ ]:
def is_equal_lst_cols(col1, col2):
  res = [x for x in col1 + col2 if x not in col1 or x not in col2]
  if not res:
    print("Списки совпаают")
  else:
    print("*** Несовпадающие элементы списков: ***")
    print('\n'.join(res))

### Функция сравнения пары колонок датафрейма на совпадение

In [ ]:
def is_equal_df_cols(df, col1, col2):
  dfcmp = np.where(df[col1] == df[col2], True, False)
  df_not_equal_indexes = df[~dfcmp].index
  print(df[~dfcmp][[col1, col2]])
#  return df_not_equal_indexes

## Исследование и преобразование исходного набора данных

### Обзор структуры и содержания данных

In [ ]:
# Размерность исходная
df.shape

(1982, 4587)

In [ ]:
# Характеристика по типам данных и их количеству
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1982 entries, 0 to 1981
Columns: 4587 entries, ДокументПрохождениеМедосмотра to Офтальмология1_ДвигательныйАппаратГлазДополнение_ЗначениеПредставление
dtypes: bool(1), datetime64[ns](43), float64(1253), object(3290)
memory usage: 69.3+ MB


#### Удаление дубликатов строк и солбцов

In [ ]:
# Количество дубликатов
len(df)-len(df.drop_duplicates())

56

In [ ]:
# Список дубликатов строк
df[df.duplicated()]

,ДокументПрохождениеМедосмотра,Клиент,КлиентДатаРождения,КлиентПол,КлиентПолПредставление,ДатаЗавершенияМедосмотра,ТипМедосмотра,ТипМедосмотраПредставление,ПсихОсвидетельствование,Контрагент,...,Оториноларингология3_Аудиометрия1_Значение,Оториноларингология3_Аудиометрия1_ЗначениеПредставление,Оториноларингология1_ЖалобыДополнение_Значение,Оториноларингология1_ЖалобыДополнение_ЗначениеПредставление,Стоматология2_ВП2_Значение,Стоматология2_ВП2_ЗначениеПредставление,Терапия2_ЗаключениеФОГК_Значение,Терапия2_ЗаключениеФОГК_ЗначениеПредставление,Офтальмология1_ДвигательныйАппаратГлазДополнение_Значение,Офтальмология1_ДвигательныйАппаратГлазДополнение_ЗначениеПредставление
392,2997d782-55ab-11ed-8108-0cc47aab8067,116fe6a5-455f-11ed-8107-0cc47aab8067,1977-03-06,Мужской,Мужской,2022-10-27,Периодический,Периодический,False,bd085484-1d16-11ed-8103-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
395,61d271b9-55ab-11ed-8108-0cc47aab8067,c547414b-732c-11e6-8360-b8975a94e496,1963-01-17,Мужской,Мужской,2022-10-27,Периодический,Периодический,False,bd085484-1d16-11ed-8103-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
396,fc62264f-55aa-11ed-8108-0cc47aab8067,116fe6a3-455f-11ed-8107-0cc47aab8067,1968-07-22,Мужской,Мужской,2022-10-27,Периодический,Периодический,False,bd085484-1d16-11ed-8103-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
398,e334cb0e-55a6-11ed-8108-0cc47aab8067,116fe6a8-455f-11ed-8107-0cc47aab8067,1971-02-22,Женский,Женский,2022-10-27,Периодический,Периодический,False,bd085484-1d16-11ed-8103-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
399,549479ee-55aa-11ed-8108-0cc47aab8067,24086b2f-4bf7-11e7-80d9-708bcd7f83cd,1976-06-16,Женский,Женский,2022-10-27,Периодический,Периодический,False,bd085484-1d16-11ed-8103-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410,832c79bc-55b5-11ed-8108-0cc47aab8067,39f65c6b-34f7-11e9-80ce-0cc47aab8067,1982-03-19,Мужской,Мужской,2022-10-27,Периодический,Периодический,False,963d16f4-d69e-11e7-80de-708bcd7f83cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411,fb21176c-55bf-11ed-8108-0cc47aab8067,5ba8b3e2-81bf-11e9-80cf-0cc47aab8067,1969-09-02,Мужской,Мужской,2022-10-27,Периодический,Периодический,False,963d16f4-d69e-11e7-80de-708bcd7f83cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
429,30b084e0-5678-11ed-8108-0cc47aab8067,9fc8140c-bd93-11e8-80ce-0cc47aab8067,1970-01-21,Женский,Женский,2022-10-28,Периодический,Периодический,False,963d16f4-d69e-11e7-80de-708bcd7f83cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
430,00d62b6d-5678-11ed-8108-0cc47aab8067,dae940bb-c84c-11e8-80ce-0cc47aab8067,1970-09-06,Женский,Женский,2022-10-28,Периодический,Периодический,False,963d16f4-d69e-11e7-80de-708bcd7f83cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
431,723358f4-568f-11ed-8108-0cc47aab8067,5aafffb4-c344-11e9-80cf-0cc47aab8067,1979-03-04,Мужской,Мужской,2022-10-28,Периодический,Периодический,False,963d16f4-d69e-11e7-80de-708bcd7f83cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Какие значения выпадают на дубликаты в целевом столбце

In [ ]:
df[df.duplicated()]['ЗаключениеМК']

392                      Годен
395                      Годен
396            ВременноНегоден
398                      Годен
399                      Годен
410                      Годен
411                      Годен
429                      Годен
430                      Годен
431                      Годен
432                      Годен
433                      Годен
435                      Годен
436                      Годен
439                      Годен
448                      Годен
449                      Годен
450                      Годен
451                      Годен
452     ГоденСКоррекциейЗрения
470     ГоденСКоррекциейЗрения
471                      Годен
496                      Годен
506                      Годен
507                      Годен
509                      Годен
510                      Годен
585     ГоденСКоррекциейЗрения
586                      Годен
681                      Годен
1085                     Годен
1086                     Годен
1087    

Практически только преобладающие (Годен)

In [ ]:
# Количество повторяющихся строк в DataFrame
len(df) - len(df.drop_duplicates())

56

In [ ]:
# Удаление дубликатов строк
df.drop_duplicates(keep='first', inplace=True)

In [ ]:
# Размер данных после удаления дубликатов строк
df.shape

(1926, 4587)

In [ ]:
1982 - 1926

56

Удалено 56 дубликатов строк

#### Проверка наличия пустых столбцов и их удаление:

In [ ]:
dn = df.isnull().sum()
print('Количество пустых столбцов:', len(dn[dn == 1926]))
df.dropna(axis=1, how='all', inplace=True)
df.shape

Количество пустых столбцов: 884


(1926, 3703)

- Далее работаем с датафреймом:
      (1926, 3703)

In [ ]:
df.info() #(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1926 entries, 0 to 1981
Columns: 3703 entries, ДокументПрохождениеМедосмотра to Офтальмология1_ДвигательныйАппаратГлазДополнение_ЗначениеПредставление
dtypes: bool(1), datetime64[ns](43), float64(369), object(3290)
memory usage: 54.4+ MB


#### Анализ заполненности столбцов

In [ ]:
df.count()

ДокументПрохождениеМедосмотра                                             1926
Клиент                                                                    1926
КлиентДатаРождения                                                        1926
КлиентПол                                                                 1926
КлиентПолПредставление                                                    1926
                                                                          ... 
Стоматология2_ВП2_ЗначениеПредставление                                      1
Терапия2_ЗаключениеФОГК_Значение                                             1
Терапия2_ЗаключениеФОГК_ЗначениеПредставление                                1
Офтальмология1_ДвигательныйАппаратГлазДополнение_Значение                    1
Офтальмология1_ДвигательныйАппаратГлазДополнение_ЗначениеПредставление       1
Length: 3703, dtype: int64

Самые заполненные столбцы, если они не информационно-учётного характера, то могут быть существенными для обучения НС

- Полностью заполненные столбцы

In [ ]:
list(df.count().index[df.count().values == 1926])

['ДокументПрохождениеМедосмотра',
 'Клиент',
 'КлиентДатаРождения',
 'КлиентПол',
 'КлиентПолПредставление',
 'ДатаЗавершенияМедосмотра',
 'ТипМедосмотра',
 'ТипМедосмотраПредставление',
 'ПсихОсвидетельствование',
 'Контрагент',
 'Профессия',
 'ВредныеФакторы']

In [ ]:
df[['КлиентПол', 'КлиентПолПредставление']]

,КлиентПол,КлиентПолПредставление
0,Женский,Женский
1,Мужской,Мужской
2,Мужской,Мужской
3,Мужской,Мужской
4,Мужской,Мужской
...,...,...
1977,Мужской,Мужской
1978,Мужской,Мужской
1979,Мужской,Мужской
1980,Женский,Женский


In [ ]:
# Проверка совпадающих колонок
is_equal_df_cols(df, 'КлиентПол', 'КлиентПолПредставление')

Empty DataFrame
Columns: [КлиентПол, КлиентПолПредставление]
Index: []


различий нет - полное совпадение

In [ ]:
# Проверка уникальности в колонках
prn_unique_txt_col(df, 'КлиентПол')

*** КлиентПол(2 уникальных): ***
Женский
Мужской



Что за ↓"Контрагенты"↓?  (177 уникальных!)



In [ ]:
prn_unique_txt_col(df, 'Контрагент')

*** Контрагент(177 уникальных): ***
23c173a7-30d9-11e7-80d8-708bcd7f83cd
a6d559b0-7835-11eb-80e8-0cc47aab8067
a1e92db1-2ea1-11e9-80ce-0cc47aab8067
1ef9e604-5432-11ec-80fc-0cc47aab8067
1d072795-48e4-11e6-82d8-d897ba0a5c60
992c8208-4e0d-11ed-8107-0cc47aab8067
e8b70525-bcef-11e8-80ce-0cc47aab8067
a495b8a2-fe8a-11ec-8103-0cc47aab8067
16552adb-a1a4-11eb-80f0-0cc47aab8067
e935abf8-4ed4-11e9-80ce-0cc47aab8067
4c79459c-b702-11e8-80ce-0cc47aab8067
ac198fbb-2568-11ea-80d7-0cc47aab8067
8cd212c1-84c9-11ec-80fc-0cc47aab8067
caa5478c-7a99-11ec-80fc-0cc47aab8067
e71ae138-fd17-11ea-80e2-0cc47aab8067
4d0f5a2d-1589-11ea-80d3-0cc47aab8067
ff5c0c64-6975-11e8-80ce-0cc47aab8067
9fe38364-b4b5-11ec-8100-0cc47aab8067
f6b57b5c-836d-11e8-80ce-0cc47aab8067
295d0691-22da-11ed-8104-0cc47aab8067
b1640169-de75-11e9-80d1-0cc47aab8067
31f0b8ac-b66a-11e8-80ce-0cc47aab8067
0ca6b915-6291-11e9-80ce-0cc47aab8067
9fd0ecba-afea-11e7-80dc-708bcd7f83cd
150475e5-8431-11e8-80ce-0cc47aab8067
226f6094-63db-11e8-80ce-0cc47aab8067
5a

In [ ]:
prn_unique_txt_col(df, 'Профессия', sort_asc = True)

*** Профессия(690 уникальных): ***
Авиамеханик
Авиамеханик 
Авиамеханик Авиамеханик
Авиатехник  ПиД
Авиатехник АиРЭО
Авиатехник ПиД
Агент-комплектовщик
Агроном 1 категории
Администратор
Администратор-кассир
Анестезиолог
Ассистент врача-стоматолога
Бармен
Барменеджер
Бригадир
Бригадир-электромонтажник по обслуживанию и ремонту электрооборудования
Бригадир-электромонтер по обслуживанию и ремонту электрооборудования
Бульдозерист
Бурильщик
Бурильщик капитального ремонта скважин
Бухгалтер
Бухгалтер 1 категории
Вальщик леса
Вальщик леса 6 разряда
Вальщик леса 6 разряда 
Ведущий агроном
Ведущий агроном по защите растений
Ведущий агроном по семеноводству
Ведущий бухгалтер
Ведущий геодезист
Ведущий геофизик
Ведущий инженер
Ведущий инженер 
Ведущий инженер (топ оборудование)
Ведущий инженер - проектировщик
Ведущий инженер АиРЭО
Ведущий инженер ПиД
Ведущий инженер по бурению 
Ведущий инженер по радиационной безопасности (класс вредности 3.1)
Ведущий инженер по сварке
Ведущий инженер-дефектоскопис

In [ ]:
# Исправление ошибок
error = {
    'Ведущий специалист по организации обуения':'Ведущий специалист по организации обучения',
    'Водитель по доставке готовй продукции':'Водитель по доставке готовой продукции',
    'Авиамеханик Авиамеханик':'Авиамеханик',
    'Водитель автомобиля 1 клаасса 5 разряда':'Водитель автомобиля 1 класса 5 разряда',
    'Водитель автомобиля- грузчик':'Водитель автомобиля-грузчик',
    'Водитель транспортного средства категории С':'Водитель транспортного средства категории "С"',
    'Водитель вездехода 6 разряд':'Водитель вездехода 6 разряда',
    'Ведущий инженер-электроник ( 5 класса)':'Ведущий инженер-электроник (5 класса)',
    'Взрывник 6 раазряда':'Взрывник 6 разряда',
    'Взрывник 6 го разряда':'Взрывник 6 разряда',
    'Водитель автомобиля (камаз-манипулятор, ломовоз':'Водитель автомобиля (камаз-манипулятор, ломовоз)',
    'Водитель автомобиля- грузчик':'Водитель автомобиля-грузчик',
    'Врач - стоматолог терапевт':'Врач стоматолог-терапевт',
    'Врач стоматолог- общей практики':'Врач стоматолог-общей практики',
    'Врач стоматолог - ортопед':'Врач стоматолог-ортодонт',
    'Врач стоматолог – детский, Врач - ортодонт':'Врач стоматолог-детский',
    'Врач-стоматолог':'Врач стоматолог',
    'Врач-стоматолог-ортопед':'Врач стоматолог-ортопед',
    'Врач-стоматолог-терапевт':'Врач стоматолог-терапевт',
    'Врач-стоматолог-хирург':'Врач стоматолог-хирург',
    'Грузчик — комплектовщик':'Грузчик-комплектовщик',
    'Дефектоскопист ренгено-гаммаграфирования НК':'Дефектоскопист ренnгено-,гаммаграфирования НК',
    'Дефектоскопист рентгено-гаммаграфирования НК':'Дефектоскопист рентгено-,гаммаграфирования НК',
    'Дефектоскопист рентгеногаммаграфирования НК':'Дефектоскопист рентгено-,гаммаграфирования НК',
    'Дефектоскопист рентгенограммаграфирования НК':'Дефектоскопист рентгено-,гаммаграфирования НК',
    'Дефектоскопист рентгеногаммаграфирования НК':'Дефектоскопист рентгено-,гаммаграфирования НК',
    'Дефектоскопист рентгенограммаграфирования НК':'Дефектоскопист рентгено-,гаммаграфирования НК',
    'Дефектоскопист ренnгено-,гаммаграфирования НК':'Дефектоскопист рентгено-,гаммаграфирования НК',
    'Инженер по буровым работам (ответственный  руководитель буровых работ)':'Инженер по буровым работам (ответственный руководитель буровых работ)',
    'Инженер по взрывным работам(ответственный руководитель взрывных работ) 1 категории':'Инженер по взрывным работам (ответственный руководитель взрывных работ) 1 категории',
    'Зав складом':'Зав. складом',
    'Зав. Склад':'Зав. cкладом',
    'Старший  инженер':'Старший инженер',
    'Старший  специалист':'Старший специалист',
    'Сушеф':'Су-шеф',
    'Старший  инженер':'Старший инженер',
    'Старшй инженер':'Старший инженер',
    'администратор ОПИР':'Администратор ОПИР',
    'ведущий инженер':'Администратор ОПИР',
    'водитель':'Водитель',
    'водитель автомобиля':'Водитель автомобиля',
    'водитель автомобиля 6 разряда':'Водитель автомобиля 6 разряда',
    'водитель вахтового автобуса':'Водитель вахтового автобуса',
    'водитель погрузчика':'Водитель погрузчика',
    'грузчик':'Грузчик',
    'грузчик-комплектовщик':'Грузчик',
    'кассир':'Кассир',
    'комплектовщик':'Комплектовщик',
    'курьер':'Курьер',
    'машинист ППДУ':'Машинист ППДУ',
    'машинист погрузчика':'Машинист погрузчика',
    'машинист экструдера':'Машинист экструдера',
    'менеджер касс':'Менеджер касс',
    'оператор станков с ЧПУ':'Оператор станков с ЧПУ',
    'педагог':'Педагог',
    'пекарь':'Пекарь',
    'повар':'Повар',
    'продавец-кассир':'Продавец-кассир',
    'продавец-консультант':'Продавец-консультант',
    'сантехник':'Сантехник',
    'старший продавец':'Старший продавец',
    'супервайзер':'Супервайзер',
    'тренер-преподаватель':'Тренер-преподаватель',
    'управление наземными транспортными средствами (кат В)':'Управление наземными транспортными средствами (кат В)',
    'фармацевт':'Фармацевт',
    'хостесс':'Хостесс',
    'Монтажник по монтажу стальных и железобетонных конструкций ':'Монтажник по монтажу стальных и железобетонных конструкций',
    'Монтажник технологических трубопроводов ':'Монтажник технологических трубопроводов',
    'Повар ':'Повар',
    'чистильщик, Монтажник по монтажу строительных и железобетоных конструкции':'Чистильщик',
    'Чистильщик, Монтажник по монтажу строительных и железобетоных конструкции':'Чистильщик',
    'Хостесс':'Хостес',
    'Электромонтажник ':'Электромонтажник',
    'Электромонтер по ремонту и обслуживанию электроборудования':'Электромонтер по ремонту и обслуживанию электрооборудования',
    'Электрогазосварщик, 5 разряд':'Электрогазосварщик 5 разряда',
    'Уборщик производственных помещений ':'Уборщик производственных помещений'
}
df['Профессия'] = df['Профессия'].replace(error)

In [ ]:
# Удаление концевых пробелов
df['Профессия'] = df['Профессия'].str.strip()

In [ ]:
prn_unique_txt_col(df, 'Профессия', sort_asc = True)

*** Профессия(600 уникальных): ***
Авиамеханик
Авиатехник  ПиД
Авиатехник АиРЭО
Авиатехник ПиД
Агент-комплектовщик
Агроном 1 категории
Администратор
Администратор ОПИР
Администратор-кассир
Анестезиолог
Ассистент врача-стоматолога
Бармен
Барменеджер
Бригадир
Бригадир-электромонтажник по обслуживанию и ремонту электрооборудования
Бригадир-электромонтер по обслуживанию и ремонту электрооборудования
Бульдозерист
Бурильщик
Бурильщик капитального ремонта скважин
Бухгалтер
Бухгалтер 1 категории
Вальщик леса
Вальщик леса 6 разряда
Ведущий агроном
Ведущий агроном по защите растений
Ведущий агроном по семеноводству
Ведущий бухгалтер
Ведущий геодезист
Ведущий геофизик
Ведущий инженер
Ведущий инженер (топ оборудование)
Ведущий инженер - проектировщик
Ведущий инженер АиРЭО
Ведущий инженер ПиД
Ведущий инженер по бурению
Ведущий инженер по радиационной безопасности (класс вредности 3.1)
Ведущий инженер по сварке
Ведущий инженер-дефектоскопист
Ведущий инженер-конструктор (5 класс)
Ведущий инженер-элек

In [ ]:
prn_unique_txt_col(df, 'ВредныеФакторы')

*** ВредныеФакторы(515 уникальных): ***
11.4
4.2.5
11.1
4.3.1,10,11.1
1.50,4.2.5,4.1,1.8.1.1
1.30.1,1.34.1,1.37.1
5.1,6.1,6.2
5.1,6
5.1,6.2,6.1
5.1
6.1,6.2,15
6.1,6.2
1.8.1.1,1.8.2.1,1.14.2,1.19.1,1.34.1,1.37.1,1.50
1.34.1,1.19.1,1.14.2,1.50,1.37.1,1.8.1.1,1.8.2.1
1.8.1.1,1.8.2.1,1.14.2,1.19.1,1.37.1,1.50,1.34.1
1.8.1.1,1.8.2.1,1.14.2,1.34.1,1.37.1,1.50,1.19.1
23
4.4
1.23,1.46,3.4
1.50,1.34.1,1.37.1
1.8.1.1,1.50
1.34.1,1.37.1,1.50
1.50,4.3.1
1.50,1.8.1.1
1.1,1.8.1.1,1.8.1.2,1.14.2,1.18.3,1.19.1,1.29.1,1.29.2,1.30.1,1.36.1,1.37.1.1,1.34.1,1.50
1.1,1.8.1.1,1.8.1.2,1.14.2,1.18.3,1.19.1,1.29.1,1.29.2,1.30.1,1.34.1,1.36.1,1.37.1,1.50
1.37.1,1.29.2,1.50,1.14.2,1.8.1.1,17,6.1
1.1,1.8.1.1,1.8.1.2,1.14.2,1.18.3,1.19.1,1.29.1,1.29.2,1.30.1,1.34.1,1.36.1,1.50,1.37.1
1.1,1.8.1.1,1.8.1.2,1.14.2,1.18.3,1.19.1,1.29.1,1.29.2,1.30.1,1.36.1,1.37.1,1.34.1,1.50
1.1,1.8.1.1,1.8.1.2,1.14.2,1.18.3,1.29.1,1.29.2,1.30.1,1.34.1,1.36.1,1.37.1,1.50,6.1
1.8.1.1,1.14.2,1.29.2,1.37.1,1.50,6.1,17
1.14.2,1.29.2,1.29.4

In [ ]:
prn_unique_txt_col(df, 'ЗаключениеМК', 'ЗаключениеМКПредставление', 'Заключение')

*** ЗаключениеМК(9 уникальных): ***
Годен
ГоденСКоррекциейЗрения
ГоденБезРаботНаВысотах
НуждаетсяВДообследованииИЛечении
ВременноНегоден
ОграниченноГоден
машинист бульдозера
Газоспасатель
 Газоспасатель

*** ЗаключениеМКПредставление(9 уникальных): ***
Годен к работе в указанных условиях
Годен к работе в указанных условиях (с коррекцией зрения)
Годен к работе в указанных условиях (без работ на высотах)
Нуждается в дообследовании и лечении
Временно не годен
Ограниченно годен
тракторист
ГоденСКоррекциейЗрения
Годен

*** Заключение(14 уникальных): ***
Годен к работе в указанных условиях
Годен к работе в указанных условиях (с коррекцией зрения)
Годен к работе в указанных условиях (с коррекцией зрения), кроме п.6.1
Годен к работе в указанных условиях (без работ на высотах) Противопоказана работа связанная с  высотой ,подьем и перемещение тяжестей ,длительной ходьбой.
Нуждается в дообследовании и лечении
Временно не годен
Ограниченно годен
Годен к работе в указанных условиях (с коррекцией зр

In [ ]:
#
count_columns_fill(df, 'ЗаключениеМК', 'ЗаключениеМКПредставление', 'Заключение')

Текущая структура датафрейма:  (1926, 3703) 

*** ЗаключениеМК: ***
Количество непустых элементов:  1519
Количество уникальных элементов:  9 

*** ЗаключениеМКПредставление: ***
Количество непустых элементов:  1519
Количество уникальных элементов:  9 

*** Заключение: ***
Количество непустых элементов:  1507
Количество уникальных элементов:  14 



#### Починим целевой столбец

##### Удаление строк с ошибками

In [ ]:
# Уникальные
prn_unique_txt_col(df, 'ЗаключениеМК')

*** ЗаключениеМК(9 уникальных): ***
Годен
ГоденСКоррекциейЗрения
ГоденБезРаботНаВысотах
НуждаетсяВДообследованииИЛечении
ВременноНегоден
ОграниченноГоден
машинист бульдозера
Газоспасатель
 Газоспасатель



In [ ]:
# Удаление строк с ошибками
values = [' Газоспасатель', 'Газоспасатель', 'машинист бульдозера']
print(df.shape)
df = df[df['ЗаключениеМК'].isin (values) == False ]
print(df.shape)

(1926, 3703)
(1922, 3703)


In [ ]:
prn_unique_txt_col(df, 'ЗаключениеМК', sort_asc = True)

*** ЗаключениеМК(6 уникальных): ***
ВременноНегоден
Годен
ГоденБезРаботНаВысотах
ГоденСКоррекциейЗрения
НуждаетсяВДообследованииИЛечении
ОграниченноГоден



##### Приводим целевой столбец с трафарету: 'Годен', 'Не годен', 'Ограниченно годен'

In [ ]:
count_columns_fill(df, 'ЗаключениеМК')

Текущая структура датафрейма:  (1922, 3703) 

*** ЗаключениеМК: ***
Количество непустых элементов:  1515
Количество уникальных элементов:  6 



In [ ]:
change = {
    'ВременноНегоден':'Не годен',
    'ГоденБезРаботНаВысотах':'Годен',
    'ГоденСКоррекциейЗрения':'Годен',
    'НуждаетсяВДообследованииИЛечении':'Не годен',
    'ОграниченноГоден':'Ограниченно годен'
}
df['ЗаключениеМК'] = df['ЗаключениеМК'].replace(change)

In [ ]:
prn_unique_txt_col(df, 'ЗаключениеМК', sort_asc = True)

*** ЗаключениеМК(3 уникальных): ***
Годен
Не годен
Ограниченно годен



In [ ]:
# Количество столбцов с заданной заполненностью
not_empty_vals_in_column = 1000
col_fill = list(df.count().index[df.count().values >= not_empty_vals_in_column])
print('Размерность датафрейма: ', df.shape)
print(f'Количество столбцов, удовлетворяющих условию >= "{not_empty_vals_in_column}" равно: ', len(col_fill))

Размерность датафрейма:  (1922, 3703)
Количество столбцов, удовлетворяющих условию >= "1000" равно:  426


Заполненность, как установлено ниже, бывает разная:
- если при уникальном = 1, то это просто формальное заполнение типа "Годен" или "Здоров", что бесполезно для обучения, имхо

## Анализ столбцов типа 'float64'

In [ ]:
# Все цифровые колонки
float_cols = df.select_dtypes(include=['float64']).columns.to_list()
float_cols

['ПсихиатрияНаркология1_Профессиональное1',
 'ПсихиатрияНаркология1_Чистая_Значение',
 'ПсихиатрияНаркология1_СоСледамиВВИнъекций_Значение',
 'ПсихиатрияНаркология1_Контактен_Значение',
 'ПсихиатрияНаркология1_НеКонтактен_Значение',
 'ПсихиатрияНаркология1_Адекватен_Значение',
 'ПсихиатрияНаркология1_НеАдекватен_Значение',
 'Офтальмология1_Профессиональное1',
 'Офтальмология1_ДискиЗрительныхНервовБледноРозовые_Значение',
 'Офтальмология1_ГраницыЧеткие_Значение',
 'Оториноларингология1_Профессиональное1',
 'Оториноларингология1_ЗаДужками_Значение',
 'Оториноларингология1_Отсутствуют_Значение',
 'Оториноларингология1_Увеличены_Значение',
 'Оториноларингология1_УвеличеныСПробками_Значение',
 'Оториноларингология1_УвеличеныСЛакунарнымНалетом_Значение',
 'ХирургНеИсп1_Профессиональное1',
 'ХирургНеИсп1_Мягкий_Значение',
 'ХирургНеИсп1_Напряженный_Значение',
 'ХирургНеИсп1_БезболезненныйПриПальпации_Значение',
 'ХирургНеИсп1_Болезненный_Значение',
 'ХирургНеИсп1_ОтрицателенСОбеихСторон_Значе

In [ ]:
# count_columns_fill(df, ['ТерапевтНеИсп1_АртериальноеДавление21_Значение', 'ТерапевтНеИсп1_АртериальноеДавление21_ЗначениеПредставление'])
count_columns_in_list_fill(df, float_cols)


Текущая структура датафрейма:  (1922, 3703) 

*** ПсихиатрияНаркология1_Профессиональное1: ***
Количество непустых элементов:  1468
Количество уникальных элементов:  1 

*** ПсихиатрияНаркология1_Чистая_Значение: ***
Количество непустых элементов:  3
Количество уникальных элементов:  1 

*** ПсихиатрияНаркология1_СоСледамиВВИнъекций_Значение: ***
Количество непустых элементов:  3
Количество уникальных элементов:  1 

*** ПсихиатрияНаркология1_Контактен_Значение: ***
Количество непустых элементов:  5
Количество уникальных элементов:  2 

*** ПсихиатрияНаркология1_НеКонтактен_Значение: ***
Количество непустых элементов:  5
Количество уникальных элементов:  1 

*** ПсихиатрияНаркология1_Адекватен_Значение: ***
Количество непустых элементов:  5
Количество уникальных элементов:  2 

*** ПсихиатрияНаркология1_НеАдекватен_Значение: ***
Количество непустых элементов:  5
Количество уникальных элементов:  1 

*** Офтальмология1_Профессиональное1: ***
Количество непустых элементов:  1169
Количест

Из 1922 записей столбцов

'ТерапевтНеИсп1_АртериальноеДавление21_Значение', 'ТерапевтНеИсп1_АртериальноеДавление21_ЗначениеПредставление'

заполнены 8 полей 2-мя уникальными значениями.

Подобная же катрина заполненности у большинства числовых столбцов:

- *** ПсихиатрияНаркология1_Профессиональное1: ***
Количество непустых элементов:  1468
Количество уникальных элементов:  1

- *** ХирургНеИсп1_НеУчащено_Значение: ***
Количество непустых элементов:  2
Количество уникальных элементов:  1

- *** Гинекология1_Профессиональное1: ***
Количество непустых элементов:  415
Количество уникальных элементов:  1
- И т.п.


*Интересны были бы для дальнейшего использования несколько столбцов:*

- *** Терапия1_П_ЗначениеПредставление: ***
Количество непустых элементов:  1666
Количество уникальных элементов:  67
- *** Терапия1_Рост_ЗначениеПредставление: ***
Количество непустых элементов:  1599
Количество уникальных элементов:  54
- *** Терапия1_Вес_ЗначениеПредставление: ***
Количество непустых элементов:  1599
Количество уникальных элементов:  92
- Открытый список...

===============

- Решение для модели первой архитектуры:
  - числовые параметры исключаются из первого подхода к обучению

## Удаляем малоинформативные (для первой модели) данные

In [ ]:
# Формирование списка сохраняемых столбцов
#
dfcols_to_save = ['КлиентДатаРождения', 'КлиентПол', 'Контрагент',
                  'Профессия', 'ЗаключениеМК', 'ВредныеФакторы']

dfcols_to_save = dfcols_to_save + list(df.columns[df.columns.str.contains('Обследование')].to_numpy())
dfcols_to_save = dfcols_to_save + list(df.columns[df.columns.str.contains('Заключение_ЗначениеПредставление')].to_numpy())
dfcols_to_save = dfcols_to_save + list(df.columns[df.columns.str.contains('ГоденНегоден_ЗначениеПредставление')].to_numpy())
dfcols_to_save = dfcols_to_save + list(df.columns[df.columns.str.contains('Диагноз_ЗначениеПредставление')].to_numpy())
dfcols_to_save = dfcols_to_save + list(df.columns[df.columns.str.contains('ПрофнепригодностьВременнаяПредставление1')].to_numpy())
dfcols_to_save = dfcols_to_save + list(df.columns[df.columns.str.contains('ПрофнепригодностьВременнаяПредставление2')].to_numpy())
dfcols_to_save = dfcols_to_save + list(df.columns[df.columns.str.contains('МКБ101')].to_numpy())

dfcols_to_save

['КлиентДатаРождения',
 'КлиентПол',
 'Контрагент',
 'Профессия',
 'ЗаключениеМК',
 'ВредныеФакторы',
 'ФункциональнаяДиагностика1_ОбследованиеПроведено_Значение',
 'ФункциональнаяДиагностика1_ОбследованиеПроведено_ЗначениеПредставление',
 'Дерматовенерология1_ОбследованиеНаПедикулез_Значение',
 'Дерматовенерология1_ОбследованиеНаПедикулез_ЗначениеПредставление',
 'ЭКГНеИсп1_ОбследованиеПроведено_Значение',
 'ЭКГНеИсп1_ОбследованиеПроведено_ЗначениеПредставление',
 'Хирургия1_ОбследованиеНаПедикулез_Значение',
 'Хирургия1_ОбследованиеНаПедикулез_ЗначениеПредставление',
 'ФункциональнаяДиагностика2_ОбследованиеПроведено_Значение',
 'ФункциональнаяДиагностика2_ОбследованиеПроведено_ЗначениеПредставление',
 'Дерматовенерология2_ОбследованиеНаПедикулез_Значение',
 'Дерматовенерология2_ОбследованиеНаПедикулез_ЗначениеПредставление',
 'Хирургия3_ОбследованиеНаПедикулез_Значение',
 'Хирургия3_ОбследованиеНаПедикулез_ЗначениеПредставление',
 'Хирургия2_ОбследованиеНаПедикулез_Значение',
 'Хиру

In [ ]:
# Анализ заполненности выбранных столбцов
count_columns_in_list_fill(df, dfcols_to_save)

Текущая структура датафрейма:  (1922, 3703) 

*** КлиентДатаРождения: ***
Количество непустых элементов:  1922
Количество уникальных элементов:  1794 

*** КлиентПол: ***
Количество непустых элементов:  1922
Количество уникальных элементов:  2 

*** Контрагент: ***
Количество непустых элементов:  1922
Количество уникальных элементов:  177 

*** Профессия: ***
Количество непустых элементов:  1922
Количество уникальных элементов:  598 

*** ЗаключениеМК: ***
Количество непустых элементов:  1515
Количество уникальных элементов:  3 

*** ВредныеФакторы: ***
Количество непустых элементов:  1922
Количество уникальных элементов:  512 

*** ФункциональнаяДиагностика1_ОбследованиеПроведено_Значение: ***
Количество непустых элементов:  430
Количество уникальных элементов:  1 

*** ФункциональнаяДиагностика1_ОбследованиеПроведено_ЗначениеПредставление: ***
Количество непустых элементов:  430
Количество уникальных элементов:  1 

*** Дерматовенерология1_ОбследованиеНаПедикулез_Значение: ***
Количе

Избавимся от столбцов, непригодных для качества обучения: с единственным выбором (количество уникальных == 1)

In [ ]:
dfcols_to_save = reduce_colomns_unique_1(df, dfcols_to_save)

Входящий размер списка столбцов:  205
Исходящий размер списка столбцов:  78


Структура заполненности оставшихся

In [ ]:
count_columns_in_list_fill(df, dfcols_to_save)

Текущая структура датафрейма:  (1922, 3703) 

*** КлиентДатаРождения: ***
Количество непустых элементов:  1922
Количество уникальных элементов:  1794 

*** КлиентПол: ***
Количество непустых элементов:  1922
Количество уникальных элементов:  2 

*** Контрагент: ***
Количество непустых элементов:  1922
Количество уникальных элементов:  177 

*** Профессия: ***
Количество непустых элементов:  1922
Количество уникальных элементов:  598 

*** ЗаключениеМК: ***
Количество непустых элементов:  1515
Количество уникальных элементов:  3 

*** ВредныеФакторы: ***
Количество непустых элементов:  1922
Количество уникальных элементов:  512 

*** УЗИДиагностика1_Заключение_ЗначениеПредставление: ***
Количество непустых элементов:  403
Количество уникальных элементов:  177 

*** УЗИДиагностика2_Заключение_ЗначениеПредставление: ***
Количество непустых элементов:  16
Количество уникальных элементов:  10 

*** ПсихиатрияНаркология1_ГоденНегоден_ЗначениеПредставление: ***
Количество непустых элементов: 

### Удаляем все, не выбранные для сохранение колонки

In [ ]:
# Удаление из датафрейма всех не выбранных для сохранения колонок
i = 0
try:
  while True:
    for col_to_save in dfcols_to_save:
      if df.columns[i] == col_to_save:
        i += 1
        continue
    else:
      df.drop(df.columns[i], axis= 1 , inplace=True)
except IndexError:
  df.shape

In [ ]:
df.shape

(1922, 78)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1922 entries, 0 to 1981
Data columns (total 78 columns):
 #   Column                                                          Non-Null Count  Dtype         
---  ------                                                          --------------  -----         
 0   КлиентДатаРождения                                              1922 non-null   datetime64[ns]
 1   КлиентПол                                                       1922 non-null   object        
 2   Контрагент                                                      1922 non-null   object        
 3   Профессия                                                       1922 non-null   object        
 4   ЗаключениеМК                                                    1515 non-null   object        
 5   ВредныеФакторы                                                  1922 non-null   object        
 6   ПсихиатрияНаркология1_МКБ101                                    1468 non-null   object  

### Удалим из оставшихся самые неинформационные столбцы с заполненностью < 10

In [ ]:
df.drop('Терапия3_МКБ101', axis= 1 , inplace=True)
df.drop('Терапия3_Диагноз_ЗначениеПредставление', axis= 1 , inplace=True)
df.drop('Терапия4_МКБ101', axis= 1 , inplace=True)
df.drop('Терапия4_Диагноз_ЗначениеПредставление', axis= 1 , inplace=True)
df.drop('ТерапевтНеИсп1_Диагноз_ЗначениеПредставление', axis= 1 , inplace=True)
df.drop('ТерапевтНеИсп1_МКБ101', axis= 1 , inplace=True)
df.drop('Эндоскопия1_МКБ101', axis= 1 , inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1922 entries, 0 to 1981
Data columns (total 71 columns):
 #   Column                                                          Non-Null Count  Dtype         
---  ------                                                          --------------  -----         
 0   КлиентДатаРождения                                              1922 non-null   datetime64[ns]
 1   КлиентПол                                                       1922 non-null   object        
 2   Контрагент                                                      1922 non-null   object        
 3   Профессия                                                       1922 non-null   object        
 4   ЗаключениеМК                                                    1515 non-null   object        
 5   ВредныеФакторы                                                  1922 non-null   object        
 6   ПсихиатрияНаркология1_МКБ101                                    1468 non-null   object  

In [ ]:
df.shape

(1922, 71)

## Оценка дисбаланса выборки

### Проверка "Ограниченно годен"

In [ ]:
df[df['ЗаключениеМК'] == 'Ограниченно годен']

,КлиентДатаРождения,КлиентПол,Контрагент,Профессия,ЗаключениеМК,ВредныеФакторы,ПсихиатрияНаркология1_МКБ101,ПсихиатрияНаркология1_ПрофнепригодностьВременнаяПредставление1,ПсихиатрияНаркология1_ГоденНегоден_ЗначениеПредставление,ПсихиатрияНаркология1_Диагноз_ЗначениеПредставление,...,Дерматовенерология2_Диагноз_ЗначениеПредставление,Терапия2_Диагноз_ЗначениеПредставление,Хирургия1_ПрофнепригодностьВременнаяПредставление2,Хирургия2_ПрофнепригодностьВременнаяПредставление2,Офтальмология2_МКБ101,Офтальмология2_ПрофнепригодностьВременнаяПредставление1,Офтальмология2_ПрофнепригодностьВременнаяПредставление2,Офтальмология2_Диагноз_ЗначениеПредставление,Терапия2_ПрофнепригодностьВременнаяПредставление2,Терапия1_Диагноз_ЗначениеПредставление
207,1991-05-31,Мужской,e8b70525-bcef-11e8-80ce-0cc47aab8067,Слесарь механосборочных работ 5 разряда,Ограниченно годен,"4.4,5.1,6.1,15",Z00.0,Годен,Годен,[Z00.0] Общий медицинский осмотр_x000D_\n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
571,1983-04-06,Мужской,f4590331-8ef1-11e7-80db-708bcd7f83cd,Инженер,Ограниченно годен,"4.2.5,6.2,9",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
len(df[df['ЗаключениеМК'] == 'Ограниченно годен'])

2

### Проверка "Не годен"

In [ ]:
df[df['ЗаключениеМК'] == 'Не годен']

,КлиентДатаРождения,КлиентПол,Контрагент,Профессия,ЗаключениеМК,ВредныеФакторы,ПсихиатрияНаркология1_МКБ101,ПсихиатрияНаркология1_ПрофнепригодностьВременнаяПредставление1,ПсихиатрияНаркология1_ГоденНегоден_ЗначениеПредставление,ПсихиатрияНаркология1_Диагноз_ЗначениеПредставление,...,Дерматовенерология2_Диагноз_ЗначениеПредставление,Терапия2_Диагноз_ЗначениеПредставление,Хирургия1_ПрофнепригодностьВременнаяПредставление2,Хирургия2_ПрофнепригодностьВременнаяПредставление2,Офтальмология2_МКБ101,Офтальмология2_ПрофнепригодностьВременнаяПредставление1,Офтальмология2_ПрофнепригодностьВременнаяПредставление2,Офтальмология2_Диагноз_ЗначениеПредставление,Терапия2_ПрофнепригодностьВременнаяПредставление2,Терапия1_Диагноз_ЗначениеПредставление
177,1987-03-06,Мужской,10bd7eb8-3586-11ed-8105-0cc47aab8067,Водитель автомобиля,Не годен,"18.2,18.1,1.1,1.39,1.50,4.3.2,5.1,11.1,11.3,11.4",Z00.0,Годен,Годен,[Z00.0] Общий медицинский осмотр_x000D_\n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182,1986-07-23,Мужской,e8b70525-bcef-11e8-80ce-0cc47aab8067,Водитель автомобиля 6 разряда,Не годен,"11.1,18.1",Z00.0,Годен,Годен,[Z00.0] Общий медицинский осмотр_x000D_\n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
375,1961-08-12,Мужской,bd085484-1d16-11ed-8103-0cc47aab8067,Переработчик РАО,Не годен,"6.2,4.1,6.2,4.1",Z00.0,Годен,Годен,[Z00.0] Общий медицинский осмотр_x000D_\n,...,[Z00.0] Общий медицинский осмотр_x000D_\n,NaN,Временно не годен,Временно не годен,H52.0,Годен,Годен,[H52.0] Гиперметропия_x000D_\n,Не годен,NaN
388,1968-07-22,Мужской,bd085484-1d16-11ed-8103-0cc47aab8067,Водитель автомобиля,Не годен,"4.1,6.2,18.1,18.2,4.1,6.2,18.1,18.2",Z00.0,Годен,Годен,[Z00.0] Общий медицинский осмотр_x000D_\n,...,[Z00.0] Общий медицинский осмотр_x000D_\n,NaN,Годен,Годен,H50.9,Годен,Годен,[H50.9] Косоглазие неуточненное_x000D_\n,Не годен,NaN
479,1969-07-14,Женский,f92e1e40-9f09-11e9-80cf-0cc47aab8067,ЛАБОРАНТ СПЕКТР.АНАЛИЗА,Не годен,"1.14.2,1.23,3.1.7",Z00.0,Годен,Годен,[Z00.0] Общий медицинский осмотр_x000D_\n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
978,1987-10-26,Мужской,c9d539ae-91a3-11e9-80cf-0cc47aab8067,Монтажник технологических трубопроводов 5 разряда,Не годен,"4.4,6.1,11",Z00.0,Годен,Годен,[Z00.0] Общий медицинский осмотр_x000D_\n,...,NaN,[K29.3] Хронический поверхностный гастрит_x000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[I11.9] Гипертензивная [гипертоническая] болез...
1228,1966-08-20,Мужской,3357cf50-775a-11eb-80e8-0cc47aab8067,Электромонтер,Не годен,"6.1,9,6.1,9",NaN,NaN,NaN,NaN,...,NaN,[I25.9] Хроническая ишемическая болезнь сердца...,Годен,Годен,Z00.0,Годен,Годен,[Z00.0] Общий медицинский осмотр_x000D_\n,Годен,[I25.9] Хроническая ишемическая болезнь сердца...
1752,1968-01-01,Мужской,bb4d4c59-1f29-11eb-80e3-0cc47aab8067,Водитель автомобиля,Не годен,"1.50,4.3.2,4.4,18.1,18.2",Z00.0,Годен,Годен,[Z00.0] Общий медицинский осмотр_x000D_\n,...,NaN,[I11.9] Гипертензивная [гипертоническая] болез...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[I11.9] Гипертензивная [гипертоническая] болез...
1764,1970-01-01,Мужской,0b15bc61-618c-11ec-80fc-0cc47aab8067,Машинист копра УГМК,Не годен,"1.50,4.3.1,4.3.2,4.4,11.1,18",Z00.0,Годен,Годен,[Z00.0] Общий медицинский осмотр_x000D_\n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[I11.9] Гипертензивная [гипертоническая] болез...
1949,1963-08-12,Мужской,36b24009-f108-11ea-80e2-0cc47aab8067,Плотник,Не годен,"4.4,6.1",Z00.0,Годен,Годен,[Z00.0] Общий медицинский осмотр_x000D_\n,...,NaN,[I11.9] Гипертензивная [гипертоническая] болез...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[I11.9] Гипертензивная [гипертоническая] болез...


In [ ]:
len(df[df['ЗаключениеМК'] == 'Не годен'])

10

In [ ]:
len(df[df['ЗаключениеМК'] == 'Годен'])

1503

In [ ]:
len(df[df['ЗаключениеМК'].isna()])

407

Дисбаланс выборки по целевому столбцу:
- Годен - 1503
- Не годен - 10
- Ограниченно годен - 2
- NaN - 407

## Балансировка датасета

Выделяем в отдельные датафреймы записи со значениями из целевого столбца "Не годен" и "Ограниченно годен"

In [ ]:
df_save = df
print(df.shape)
df_ng = df[df['ЗаключениеМК'] == 'Не годен']
df = df[df['ЗаключениеМК'] != 'Не годен']
print(df.shape)

(1922, 71)
(1912, 71)


In [ ]:
df_og = df[df['ЗаключениеМК'] == 'Ограниченно годен']
df = df[df['ЗаключениеМК'] != 'Ограниченно годен']
print(df.shape)

(1910, 71)


In [ ]:
len(df[df['ЗаключениеМК'].isna()])

407

In [ ]:
# Сохраняем сокращённую копию df
df_save_2 = df

In [ ]:
df_ng

,КлиентДатаРождения,КлиентПол,Контрагент,Профессия,ЗаключениеМК,ВредныеФакторы,ПсихиатрияНаркология1_МКБ101,ПсихиатрияНаркология1_ПрофнепригодностьВременнаяПредставление1,ПсихиатрияНаркология1_ГоденНегоден_ЗначениеПредставление,ПсихиатрияНаркология1_Диагноз_ЗначениеПредставление,...,Дерматовенерология2_Диагноз_ЗначениеПредставление,Терапия2_Диагноз_ЗначениеПредставление,Хирургия1_ПрофнепригодностьВременнаяПредставление2,Хирургия2_ПрофнепригодностьВременнаяПредставление2,Офтальмология2_МКБ101,Офтальмология2_ПрофнепригодностьВременнаяПредставление1,Офтальмология2_ПрофнепригодностьВременнаяПредставление2,Офтальмология2_Диагноз_ЗначениеПредставление,Терапия2_ПрофнепригодностьВременнаяПредставление2,Терапия1_Диагноз_ЗначениеПредставление
177,1987-03-06,Мужской,10bd7eb8-3586-11ed-8105-0cc47aab8067,Водитель автомобиля,Не годен,"18.2,18.1,1.1,1.39,1.50,4.3.2,5.1,11.1,11.3,11.4",Z00.0,Годен,Годен,[Z00.0] Общий медицинский осмотр_x000D_\n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182,1986-07-23,Мужской,e8b70525-bcef-11e8-80ce-0cc47aab8067,Водитель автомобиля 6 разряда,Не годен,"11.1,18.1",Z00.0,Годен,Годен,[Z00.0] Общий медицинский осмотр_x000D_\n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
375,1961-08-12,Мужской,bd085484-1d16-11ed-8103-0cc47aab8067,Переработчик РАО,Не годен,"6.2,4.1,6.2,4.1",Z00.0,Годен,Годен,[Z00.0] Общий медицинский осмотр_x000D_\n,...,[Z00.0] Общий медицинский осмотр_x000D_\n,NaN,Временно не годен,Временно не годен,H52.0,Годен,Годен,[H52.0] Гиперметропия_x000D_\n,Не годен,NaN
388,1968-07-22,Мужской,bd085484-1d16-11ed-8103-0cc47aab8067,Водитель автомобиля,Не годен,"4.1,6.2,18.1,18.2,4.1,6.2,18.1,18.2",Z00.0,Годен,Годен,[Z00.0] Общий медицинский осмотр_x000D_\n,...,[Z00.0] Общий медицинский осмотр_x000D_\n,NaN,Годен,Годен,H50.9,Годен,Годен,[H50.9] Косоглазие неуточненное_x000D_\n,Не годен,NaN
479,1969-07-14,Женский,f92e1e40-9f09-11e9-80cf-0cc47aab8067,ЛАБОРАНТ СПЕКТР.АНАЛИЗА,Не годен,"1.14.2,1.23,3.1.7",Z00.0,Годен,Годен,[Z00.0] Общий медицинский осмотр_x000D_\n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
978,1987-10-26,Мужской,c9d539ae-91a3-11e9-80cf-0cc47aab8067,Монтажник технологических трубопроводов 5 разряда,Не годен,"4.4,6.1,11",Z00.0,Годен,Годен,[Z00.0] Общий медицинский осмотр_x000D_\n,...,NaN,[K29.3] Хронический поверхностный гастрит_x000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[I11.9] Гипертензивная [гипертоническая] болез...
1228,1966-08-20,Мужской,3357cf50-775a-11eb-80e8-0cc47aab8067,Электромонтер,Не годен,"6.1,9,6.1,9",NaN,NaN,NaN,NaN,...,NaN,[I25.9] Хроническая ишемическая болезнь сердца...,Годен,Годен,Z00.0,Годен,Годен,[Z00.0] Общий медицинский осмотр_x000D_\n,Годен,[I25.9] Хроническая ишемическая болезнь сердца...
1752,1968-01-01,Мужской,bb4d4c59-1f29-11eb-80e3-0cc47aab8067,Водитель автомобиля,Не годен,"1.50,4.3.2,4.4,18.1,18.2",Z00.0,Годен,Годен,[Z00.0] Общий медицинский осмотр_x000D_\n,...,NaN,[I11.9] Гипертензивная [гипертоническая] болез...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[I11.9] Гипертензивная [гипертоническая] болез...
1764,1970-01-01,Мужской,0b15bc61-618c-11ec-80fc-0cc47aab8067,Машинист копра УГМК,Не годен,"1.50,4.3.1,4.3.2,4.4,11.1,18",Z00.0,Годен,Годен,[Z00.0] Общий медицинский осмотр_x000D_\n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[I11.9] Гипертензивная [гипертоническая] болез...
1949,1963-08-12,Мужской,36b24009-f108-11ea-80e2-0cc47aab8067,Плотник,Не годен,"4.4,6.1",Z00.0,Годен,Годен,[Z00.0] Общий медицинский осмотр_x000D_\n,...,NaN,[I11.9] Гипертензивная [гипертоническая] болез...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[I11.9] Гипертензивная [гипертоническая] болез...


In [ ]:
df_og

,КлиентДатаРождения,КлиентПол,Контрагент,Профессия,ЗаключениеМК,ВредныеФакторы,ПсихиатрияНаркология1_МКБ101,ПсихиатрияНаркология1_ПрофнепригодностьВременнаяПредставление1,ПсихиатрияНаркология1_ГоденНегоден_ЗначениеПредставление,ПсихиатрияНаркология1_Диагноз_ЗначениеПредставление,...,Дерматовенерология2_Диагноз_ЗначениеПредставление,Терапия2_Диагноз_ЗначениеПредставление,Хирургия1_ПрофнепригодностьВременнаяПредставление2,Хирургия2_ПрофнепригодностьВременнаяПредставление2,Офтальмология2_МКБ101,Офтальмология2_ПрофнепригодностьВременнаяПредставление1,Офтальмология2_ПрофнепригодностьВременнаяПредставление2,Офтальмология2_Диагноз_ЗначениеПредставление,Терапия2_ПрофнепригодностьВременнаяПредставление2,Терапия1_Диагноз_ЗначениеПредставление
207,1991-05-31,Мужской,e8b70525-bcef-11e8-80ce-0cc47aab8067,Слесарь механосборочных работ 5 разряда,Ограниченно годен,"4.4,5.1,6.1,15",Z00.0,Годен,Годен,[Z00.0] Общий медицинский осмотр_x000D_\n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
571,1983-04-06,Мужской,f4590331-8ef1-11e7-80db-708bcd7f83cd,Инженер,Ограниченно годен,"4.2.5,6.2,9",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_og

,КлиентДатаРождения,КлиентПол,Контрагент,Профессия,ЗаключениеМК,ВредныеФакторы,ПсихиатрияНаркология1_МКБ101,ПсихиатрияНаркология1_ПрофнепригодностьВременнаяПредставление1,ПсихиатрияНаркология1_ГоденНегоден_ЗначениеПредставление,ПсихиатрияНаркология1_Диагноз_ЗначениеПредставление,...,Дерматовенерология2_Диагноз_ЗначениеПредставление,Терапия2_Диагноз_ЗначениеПредставление,Хирургия1_ПрофнепригодностьВременнаяПредставление2,Хирургия2_ПрофнепригодностьВременнаяПредставление2,Офтальмология2_МКБ101,Офтальмология2_ПрофнепригодностьВременнаяПредставление1,Офтальмология2_ПрофнепригодностьВременнаяПредставление2,Офтальмология2_Диагноз_ЗначениеПредставление,Терапия2_ПрофнепригодностьВременнаяПредставление2,Терапия1_Диагноз_ЗначениеПредставление
207,1991-05-31,Мужской,e8b70525-bcef-11e8-80ce-0cc47aab8067,Слесарь механосборочных работ 5 разряда,Ограниченно годен,"4.4,5.1,6.1,15",Z00.0,Годен,Годен,[Z00.0] Общий медицинский осмотр_x000D_\n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
571,1983-04-06,Мужской,f4590331-8ef1-11e7-80db-708bcd7f83cd,Инженер,Ограниченно годен,"4.2.5,6.2,9",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Формируем сбалансированный датасет dfb:
- копируем в новый датасет записи поочерёдно из
  - df
  - df_ng
  - df_og
- дефицитные записи циклически повторяются

In [ ]:
dfb = pd.DataFrame(columns = df.columns) # Пустой датафрейм сбалансированный
j, k = 0, 0
for i in range (df.shape[0]):
  dfb.loc[len(dfb.index )] = df.iloc[i]
##
  if j < df_ng.shape[0]:
    dfb.loc[len(dfb.index )] = df_ng.iloc[j]
    j += 1
  else:
    dfb.loc[len(dfb.index )] = df_ng.iloc[0]
    j = 1
##
  if k < df_og.shape[0]:
    dfb.loc[len(dfb.index )] = df_og.iloc[k]
    k += 1
  else:
    dfb.loc[len(dfb.index )] = df_og.iloc[0]
    k = 1

In [ ]:
dfb.shape

(5730, 71)

In [ ]:
len(df_save[df_save['ЗаключениеМК'].isna()])

407

dfb - балансированный датасет, содежащий 407 записей, у которых в целевом столбце не задано значение (NaN)

В дальнейшей работе все 'NaN' переименуются в 'Не определено'.

В этом случае на выходе НС будет 4 значения:
  - Годен
  - Ограниченно годен
  - Не годен
  - Недостаточно данных для заключения

Однако, для работы строго по тех.заданию подготовим ещё 2 датасета:
- df0 - с удалёнными записями с незаданными заключениями
- dfb0 - то же самое, но сбалансированный по тому же алгоритму

In [ ]:
df = df_save

In [ ]:
df.shape

(1922, 71)

In [ ]:
len(df[df['ЗаключениеМК'].isna()])

407

In [ ]:
df_ng.shape

(10, 71)

In [ ]:
df_og.shape

(2, 71)

In [ ]:
df = df[df['ЗаключениеМК'] != 'Не годен']
df = df[df['ЗаключениеМК'] != 'Ограниченно годен']
print(df.shape)

(1910, 71)


In [ ]:
len(df[df['ЗаключениеМК'].isna()])

407

In [ ]:
df = df[~df['ЗаключениеМК'].isna()]
len(df[df['ЗаключениеМК'].isna()])

0

In [ ]:
df0 = df
df0.shape

(1503, 71)

In [ ]:
dfb0 = pd.DataFrame(columns = df0.columns) # Пустой датафрейм сбалансированный
j, k = 0, 0
for i in range (df0.shape[0]):
  dfb0.loc[len(dfb0.index )] = df0.iloc[i]
##
  if j < df_ng.shape[0]:
    dfb0.loc[len(dfb0.index )] = df_ng.iloc[j]
    j += 1
  else:
    dfb0.loc[len(dfb0.index )] = df_ng.iloc[0]
    j = 1
##
  if k < df_og.shape[0]:
    dfb0.loc[len(dfb0.index )] = df_og.iloc[k]
    k += 1
  else:
    dfb0.loc[len(dfb0.index )] = df_og.iloc[0]
    k = 1

dfb0.shape

(4509, 71)

In [ ]:
df_save.shape

(1922, 71)

In [ ]:
dfb.shape

(5730, 71)

In [ ]:
df0.shape

(1503, 71)

In [ ]:
dfb0.shape

(4509, 71)

## Сохраняем отработанный начальный и балансированный датафреймы

In [ ]:
df_save.to_excel('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/ICdata_I(71).xlsx', index=False)
dfb.to_excel('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/ICdata_I(71)b.xlsx', index=False)
df0.to_excel('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/ICdata_I(71)0.xlsx', index=False)
dfb0.to_excel('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/ICdata_I(71)0b.xlsx', index=False)



~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~